# Headers and initialising

In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import errno
from urllib.parse import urlparse
from cellpose import core, utils, io, models, metrics, dynamics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
from cellpose.dynamics import masks_to_flows
from cellpose import models
logger_setup();

>>> GPU activated? YES
2022-10-18 09:46:46,053 [INFO] WRITING LOG OUTPUT TO /home/pablo/.cellpose/run.log


# User variables

In [2]:
channel = [0,0] ## Greyscale, no nuclei
files_path = '/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc_OnlyWounds/'
inputDir = [files_path + 'StacksToSegment/']
#inputDir = ['/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/Datasets/ToProcess/RiciBarrientos/NubG4-UASmyrGFP_Control',
#            files_path + 'StacksToSegment/']
segmentedOtherDir = [files_path + 'MasksSegmentedWithOtherSoftware/'];

use_GPU = 1;
initial_model = 'cyto2'

# Transform 3D data to 2D for users to correct

## Stack to image sequence

In [4]:
try:
    os.mkdir(files_path + 'ImageSequence2D/')
except OSError as exc:
    print('Dir exists')

for currentDir in inputDir:
    print(currentDir)
    for root, subdirs, files in os.walk(currentDir):
        for filename in files:
            if filename.endswith('.tif'):
                img = io.imread(root + '/' + filename)
                for numZ in range(0, len(img)):
                    io.imsave(files_path + 'ImageSequence2D/' + filename.replace('.tif', '') + '_' + "{:03d}".format(numZ) + '.tif', img[numZ, :, :])

Dir exists
/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc_OnlyWounds/StacksToSegment/
2022-10-18 09:52:52,107 [INFO] reading tiff with 100 planes


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 844.56it/s]

2022-10-18 09:52:52,296 [INFO] reading tiff with 100 planes



100%|████████████████████████████████████████| 100/100 [00:00<00:00, 955.62it/s]


2022-10-18 09:52:52,507 [INFO] reading tiff with 99 planes


...

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 936.77it/s]


2022-10-18 09:52:55,920 [INFO] reading tiff with 100 planes


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 838.44it/s]

2022-10-18 09:52:56,093 [INFO] reading tiff with 99 planes



100%|██████████████████████████████████████████| 99/99 [00:00<00:00, 946.85it/s]


In [45]:
allFileNames = list()
for currentDir in segmentedOtherDir:
    print(currentDir)
    for root, subdirs, files in os.walk(currentDir):
        for filename in files:
            if filename.endswith('.tif'):
                currentFileName = root + '/' + filename;
                #imgs_filename = currentFileName.replace('MasksSegmentedWithOtherSoftware', 'StacksToSegment').replace('_masks', '')
                #img = io.imread(imgs_filename);
                mask = io.imread(currentFileName)
                
                for numZ in range(0, len(img)):
                    currentZFileName = filename.replace('.tif', '').replace('_masks', '') + '_' + "{:03d}".format(numZ) + '_masks.tif'
                    #io.imsave(files_path + 'ImageSequence2D/' + currentZFileName, mask)
                    allFileNames.append(currentZFileName)

/media/pablo/d7c61090-024c-469a-930c-f5ada47fb049/PabloVicenteMunuera/CellShape3DAnalysis/2_Segmentation/cellpose/WingDisc_OnlyWounds/MasksSegmentedWithOtherSoftware/
2022-10-18 14:02:27,372 [INFO] reading tiff with 99 planes


100%|███████████████████████████████████████████| 99/99 [00:01<00:00, 87.93it/s]

2022-10-18 14:02:28,518 [INFO] reading tiff with 99 planes



100%|███████████████████████████████████████████| 99/99 [00:01<00:00, 88.51it/s]

2022-10-18 14:02:29,667 [INFO] reading tiff with 99 planes



100%|███████████████████████████████████████████| 99/99 [00:01<00:00, 84.18it/s]


# Train a new model with your data

## Divide between train and test data

In [50]:
# import modules
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# split the dataset
train_data, test_data = train_test_split(allFileNames, test_size=0.1, random_state=0)



## Retrain the new model

In [ ]:
model_path = files_path + '/models/'

# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)

# set channels
channels = [chan, chan2]

# get files
output = io.load_train_test_data(train_dir, test_dir, mask_filter='_seg.npy')
train_data, train_labels, _, test_data, test_labels, _ = output

new_model_path = model.train(train_data, train_labels, 
                              test_data=test_data,
                              test_labels=test_labels,
                              channels=channels, 
                              save_path=train_dir, 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=8,
                              model_name=model_name)

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

# Using the new model created to predict 3D data

In [ ]:
# gets image files in dir (ignoring image files ending in _masks)
files = io.get_image_files(files_path, '_masks')
print(files)
images = [io.imread(f) for f in files]

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter if user diameter is 0
diameter = model.diam_labels if diameter==0 else diameter

# * with 3D you have to set the diameter manually (no auto detect) *

### TWO WAYS TO RUN CELLPOSE IN 3D

# 1. computes flows from 2D slices and combines into 3D flows to create masks
print('running cellpose 2D slice flows => masks')
masks, flows, styles, _ = model.eval(images, diameter=30, do_3D=True, 
                                     channels=[chan, chan2], 
                                     flow_threshold=flow_threshold,
                                     cellprob_threshold=cellprob_threshold)

# 2. computes masks in 2D slices and stitches masks in 3D based on mask overlap
print('running cellpose 2D + stitching masks')
masks_stitched, flows_stitched, styles_stitched, _ = model.eval(images, channels=[chan, chan2], diameter=diameter, do_3D=False, stitch_threshold=0.5)

In [ ]:
# DISPLAY RESULTS 3D flows => masks
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)

In [ ]:
# DISPLAY RESULTS stitching
plt.figure(figsize=(15,3))
for i,iplane in enumerate(np.arange(0,75,10,int)):
  img0 = plot.image_to_rgb(img_3D[iplane, [1,0]].copy(), channels=[2,3])
  plt.subplot(1,8,i+1)
  outlines = utils.masks_to_outlines(masks_stitched[iplane])
  outX, outY = np.nonzero(outlines)
  imgout= img0.copy()
  imgout[outX, outY] = np.array([255,75,75])
  plt.imshow(imgout)
  plt.title('iplane = %d'%iplane)

In [ ]:
from cellpose import io

io.masks_flows_to_seg(images, 
                      masks, 
                      flows, 
                      diameter*np.ones(len(masks)), 
                      files, 
                      channels)